In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import glob as glob
import os, errno
from scipy.optimize import curve_fit
from scipy.spatial import ConvexHull
import shutil as shutil

In [ ]:
#sort loose files into folders
def sortfiles(cdir):
    destdir = os.path.join(cdir)
    if not os.path.exists(destdir):
        os.mkdir(destdir)
    imgfiles = glob.glob(cdir+'/*.tif')
    for i in range(len(imgfiles)):
        filename = os.path.basename(imgfiles[i])
        fullfilename = imgfiles[i]
        foldername = os.path.basename(imgfiles[i]).split('_')[1]
        path = os.path.join(destdir,foldername)
        if not os.path.exists(path):
            os.mkdir(path)
        destpath = os.path.join(path,filename)
        os.rename(fullfilename,os.path.join(path,filename))

In [ ]:
#function to fit travelling wave
def travellingwave(x, a, c):
    return 1 / (1 + np.exp((x-c)/(2*a)))

In [ ]:
#function to get area of a set of points given a convex hull
def area_np(hul,img):
    my_img = np.zeros((img.shape[0], img.shape[1]), dtype = "uint8")
    pts = hul.reshape((-1,1,2))
    cv2.polylines(my_img,[pts],True,255)
    cv2.fillPoly(my_img,[pts],255)
    area = sum(my_img.reshape(img.shape[0]*img.shape[1])/255)
    return area

In [ ]:
#get fistance of each pixel from center
def distancecalc(gimg,cent):
    totaldist = 0
    maxdist = -1
    count = 0
    for i in range(gimg.shape[0]):
        for j in range(gimg.shape[1]):
            
            if gimg[i,j] != 0:
                dist = np.sqrt((i-cent[1])**2 + (j-cent[0])**2)
                if dist > 500:
                    continue
                totaldist = dist+totaldist
                count = count+1
                if dist > maxdist:
                    maxdist = dist
    if count != 0:
        return [maxdist, totaldist/count]
    else:
        return [-1,-1]

In [ ]:
#get the convex hull and the get the area of this
def hullarea(img):
    _,contours,_  = cv2.findContours(img,cv2.RETR_EXTERNAL,2)
    if(len(contours)>0):
        out = np.concatenate(contours)
        hullpoints = cv2.convexHull(out)
        p = np.squeeze(hullpoints)
        q = area_np(p,img)
        count = 0
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i,j] != 0:
                    count = count+1
    else:
        return[-1,-1]
    return [q,count/q]

In [ ]:
sortfiles(os.path.join('PATH'))

In [ ]:
basepath = 'Image_Path'
assay = "Assay_ID"
cell = 'Cell_line'
expID = "Exp_ID"
grfolders = os.path.join(basepath)
maxdiameter = 600 #diameter determined by max distance invaded
grsavefile = open(os.path.join(basepath,'OUTPUT_FILE.txt'),"w+")

#write a header to the output
writethis = "Assay" + "\t" + "ID" + "\t"+ "Exp" +"\t"+ "Well" + "\t" + "Num" + "\t" + "Time" + "\t" + "Diffusion" + "\t" + "Front" + "\t" + "Area" + "\t" + "Density" + "\t" + "Max" + "\t" + "Avg" + '\n'
grsavefile.write(writethis)

#analysis of images
imgfolders = sorted(glob.glob(os.path.join(basepath,'*')))
for i in range((len(imgfolders))):
    wellID = (os.path.join(imgfolders[i])).split('/')[-1]
        
    imgfiles = sorted(glob.glob(os.path.join(imgfolders[i],'*png')))
    
    norm = 15.30474 # take from previous analysis (average distance invaded in a pure culture at time 0 for the assay)
    for j in range(len(imgfiles)):
        
        #based on file naming convention one of these are used
        #time = round(float(imgfiles[j][-13:-11])*24 + float(imgfiles[j][-10:-8]) + float(imgfiles[j][-7:-5])/60)
        time = round(float(imgfiles[j][-18:-16])*24 + float(imgfiles[j][-15:-13]) + float(imgfiles[j][-12:-10])/60)

        #load image
        img = cv2.imread(imgfiles[j],0)
        img = img.astype('uint8')
            
        #first image per well is used for determination of the center point
        if(j == 0):
            nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
            sizes = stats[:, -1]
            max_label = 1
            if len(sizes) == 1:
                continue
            max_size = sizes[max_label]
            if(sizes.shape[0] > 2):
                for m in range(2,nb_components):
                    if sizes[m] > max_size:
                        max_label = m
                        max_size = sizes[max_label]
            cent = tuple(centroids[max_label,:].astype(int))
        
        [area,density] = hullarea(img)
        [maxextent,avgextent] = distancecalc(img,cent)
            
        polarimg = cv2.linearPolar(img,cent,1536,cv2.WARP_FILL_OUTLIERS)

        xdata = np.arange(0,maxdiameter)
        ydata = np.zeros(len(xdata))
        for xx in range(0,len(xdata)):
            ydata[xx] = np.mean(polarimg[:,xx])/255
        xdata = xdata/norm
        try:
            popt, pcov = curve_fit(travellingwave, xdata, ydata)
            
        except RuntimeError:
            popt = np.zeros(2)
            popt[0] = np.nan
            popt[1] = np.nan
            print("Error - curve_fit failed") 

        
        writethis = str(assay) + "\t" + str(i) + "\t"+str(expID) +"\t"+ str(wellID)[0] + "\t" + str(wellID)[1] + "\t" + str(time) + "\t" + str(popt[0]) + "\t" + str(popt[1]) + "\t" + str(area) + "\t" + str(density) + "\t" + str(maxextent) + "\t" + str(avgextent) + '\n'
        grsavefile.write(writethis)
            
        grsavefile.flush()
grsavefile.close()        




